## Setup

In [ ]:
import tweepy
import pandas as pd
import numpy as np

In [ ]:
user_fields=["id",
"id_str",
"name",
"screen_name",
"location",
"description",
"url",
"protected",
"followers_count",
"friends_count",
"listed_count",
"created_at",
"favourites_count",
"verified",
"statuses_count",
"has_extended_profile",
"default_profile",
"default_profile_image",
"withheld_in_countries",
"follower_of"]

## Authentication

In [ ]:
bearer_token = "<enter-bearer-token>"
auth = tweepy.OAuth2BearerHandler(bearer_token)
api = tweepy.API(auth, wait_on_rate_limit=True)

## Load data

In [ ]:
file_name = "hsd.csv"
hs_df = pd.read_csv(file_name)

## Get followers

In [ ]:
start_user_index = 200
end_user_index = 399

In [ ]:
user_list = hs_df.iloc[start_user_index:end_user_index + 1, 1] # get userIds (column 1)

follower_df = pd.DataFrame()

for userID in user_list:
    follower_list = []

    try:
        for page in tweepy.Cursor(api.get_followers, user_id=userID, count=200).pages():
            follower_list.extend(page)
        
        follower_of_user_df = pd.DataFrame(data=[ follower._json for follower in follower_list])
        follower_of_user_df['follower_of'] = userID
        
        follower_df = pd.concat([follower_df, follower_of_user_df])
    
    except tweepy.TweepyException:
        print(userID)
        print("error")
        continue

## Save relevant data as csv

In [ ]:
short_follower_df = follower_df[user_fields]
short_follower_df = short_follower_df.astype({'id': np.int64, 'has_extended_profile': bool})
short_follower_df.to_csv("hsd_follower_" + str(start_user_index) + "-" + str(end_user_index) + ".csv", index=False)